In [127]:
import numpy as np

In [128]:
import pandas as pd

In [129]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [130]:
# print(movies.head(1))

In [131]:
movies=movies.merge(credits,on='title')

In [132]:
#genre
# id
# keywords
#title 
#overview
# popularity
#cast
#crew

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]



In [ ]:
# movies=pd.DataFrame(movies)
movies.head()

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [136]:
movies.dropna(inplace=True)

In [137]:
# movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [139]:
# movies.iloc[0].genres

In [140]:
import ast
def convert(obj):
  L=[]
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [141]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)


In [142]:
import ast
def convert1(obj):
  L=[]
  c=0
  for i in ast.literal_eval(obj):
    if c!=3:
       L.append(i['name'])
       c+=1
    else:
      break
  return L

In [143]:
movies['cast']=movies['cast'].apply(convert1)

In [ ]:
movies.head()

In [ ]:
print(movies['crew'][0])

In [146]:
import ast
def convert2(obj):
  L=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
       L.append(i['name'])
       break
  return L

In [147]:
movies['crew']=movies['crew'].apply(convert2)

In [ ]:
movies.head()

In [149]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [150]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [152]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']


In [153]:
df=movies.drop(columns=['overview','genres','keywords','cast','crew'])


In [154]:
df['tags']=df['tags'].apply(lambda x:" ".join(x))

In [ ]:
df.head()

In [156]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [157]:
df['tags']=df['tags'].apply(stem) 

In [158]:
df['tags']=df['tags'].apply(lambda x:x.lower())

In [ ]:
df.head()

In [160]:
from sklearn.feature_extraction.text import CountVectorizer


In [161]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [162]:
vectors=cv.fit_transform(df['tags']).toarray()

In [ ]:
cv.get_feature_names()

In [164]:
from sklearn.metrics.pairwise import cosine_similarity

In [165]:
similarity=cosine_similarity(vectors)

In [ ]:
df.head()

In [ ]:
def recommend(movie):
  if df[df['title']==movie].shape==(0,3):
    return "wrong movie name"
  movie_index=df[df['title']==movie].index[0]
  distances=similarity[movie_index]
  movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movie_list:
    print(df.iloc[i[0]].title)
  return 
recommend('Batman Begins')

In [168]:
import pickle
pickle.dump(df,open('movies.pkl','wb'))

In [169]:
pickle.dump(similarity,open('metrix.pkl','wb'))